In [1]:
!pip install yargy
!pip install wikipedia

     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 7.1MB 8.1MB/s 
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=746dcfb889dee7acab813f53decb0359a54d431d936a159ff420ade82e02a96b
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import wikipedia
wikipedia.set_lang("ru")

from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd
from tqdm import tqdm_notebook
from gensim import utils

import warnings
warnings.filterwarnings('ignore')

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [0]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [8]:
import pandas as pd

CHARGE_NUMBER = 18

df = pd.read_csv("/content/gdrive/My Drive/lab2_words.csv")
charge = df.loc[df['IN CHARGE NUMBER'] == CHARGE_NUMBER]
charge.head(5)

,Заполнено,Unnamed: 1,IN CHARGE NUMBER,IN CHARGE NAME,HYPONYM,HYPERNYM,HYPERNYM.1,HYPERNYM.2,HYPERNYM.3,HYPERNYM.4,HYPERNYM.5,HYPERNYM.6,HYPERNYM.7,HYPERNYM.8,HYPERNYM.9,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,#ERROR!
408,NaN,409,18,Bekarys Kuan,ЗВЕРОВОДСТВО,ЖИВОТНОВОДСТВА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
409,NaN,410,18,Bekarys Kuan,ЗДРАВИЦА,ТОСТ,РЕЧЬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
410,NaN,411,18,Bekarys Kuan,ЗЕМЛЕРОЙКА,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,NaN,412,18,Bekarys Kuan,ЗЕМСНАРЯД,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,NaN,413,18,Bekarys Kuan,ЗЕРНОВКА,ПЛОД,ЖУК,НАСЕКОМОЕ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
for index, row in charge.iterrows():
    try:
      print(row['HYPONYM'])
      get_hyperonyms(row['HYPONYM'])
      print("-"*50)
    except: 
        pass

ЗВЕРОВОДСТВО
Звероводство (с конца XX века также используется словосочетание «меховая индустрия» — буквальныи перевод с англииского) — отрасль животноводства по разведению в неволе ценных пушных звереи для получения шкурок. Объекты звероводства — норка, голубои песец, серебристо-черная лисица, нутрия, соболь, речнои бобр, шиншиллы и другие животные.
С 1960-х годов отрасль стала подвергаться резкои критике борцов за права животных и противников натурального меха.
--------------------------------------------------
ЗДРАВИЦА
Здравица — кантата для смешанного хора и симфонического оркестра С. С. Прокофьева, посвященная И. В. Сталину. Авторы текста неизвестны.
--------------------------------------------------
ЗЕМЛЕРОЙКА
Землероиковые (лат. Soricidae) — семеиство млекопитающих из отряда насекомоядных.
--------------------------------------------------
ЗЕМСНАРЯД
Земснаряд — судно технического флота, предназначенное для производства дноуглубительных работ и добычи нерудных строительных материа